In [ ]:
#This is a Python implementation of the TDA Mapper algorithm for visualization of high-dimensional data. 
#For complete documentation, see https://kepler-mapper.scikit-tda.org.
pip install kmapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 12.4 MB/s eta 0:00:00


In [ ]:
#Uniform Manifold Approximation and Projection for Dimension Reduction
#https://umap-learn.readthedocs.io/en/latest/
!pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=a5fa004c335e40464001cef35c0e37b4efd33fef52ee97d45b8505f684d61601
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55640 sha256=25f36085335b6662ce546154d468837afbe93b7a9d9038e31e00eae74dc1d3a2
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


In [ ]:
import yfinance as yf
import kmapper as km
from kmapper.jupyter import display
import umap #Uniform Manifold Approximation and Projection for Dimension Reduction
import sklearn
import sklearn.manifold as manifold #https://scikit-learn.org/stable/modules/manifold.html
import matplotlib.pyplot as plt
import numpy as np



El índice Standard & Poor's 500 (Standard & Poor's 500 Index), también conocido como S&P 500, es uno de los índices bursátiles más importantes de Estados Unidos. Al S&P 500 se lo considera el índice más representativo de la situación real del mercado. Se trabajará un archivo de ticketmaster.


# `Obtengamos los datos`

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Leer el archivo
filename = open("/content/gdrive/MyDrive/Colab Notebooks/SP500_tickernames.txt", "r")
raw_tickernames = filename.read()
ticker_names = raw_tickernames.split("\n")
ticker_names = ticker_names[:len(ticker_names)-1]

In [ ]:
# Definir el rango de fechas
start_date_string = "2020-01-01"
end_date_string = "2022-04-02"

# Obtengamos los datos históricos
raw_data = yf.download(ticker_names, start=start_date_string, end=end_date_string)

[*********************100%***********************]  495 of 495 completed

16 Failed downloads:
- FB: No timezone found, symbol may be delisted
- NLSN: No timezone found, symbol may be delisted
- VIAC: No timezone found, symbol may be delisted
- ANTM: No timezone found, symbol may be delisted
- BLL: No timezone found, symbol may be delisted
- INFO: No timezone found, symbol may be delisted
- XLNX: No timezone found, symbol may be delisted
- DISCK: No timezone found, symbol may be delisted
- PBCT: No timezone found, symbol may be delisted
- DISCA: No timezone found, symbol may be delisted
- CTXS: No timezone found, symbol may be delisted
- CERN: No timezone found, symbol may be delisted
- DRE: No timezone found, symbol may be delisted
- NLOK: No timezone found, symbol may be delisted
- FBHS: No timezone found, symbol may be delisted
- TWTR: No timezone found, symbol may be delisted


In [ ]:
# obtenga precios de cierre diarios y elimine las columnas que faltan
df_close = raw_data['Adj Close'].dropna(axis='columns')

In [ ]:
# convierta pandas dataframe a numpy array, estandaricemos los datos y transpongamos el array
data = df_close.to_numpy()
data = data-np.mean(data, axis=0)/np.std(data, axis=0)
data = data.transpose()

(479, 568)

In [ ]:
# calcula el porcentaje de retorno (de la inversión) de cada ticker sobre el rango de fechas
per_return = (df_close.to_numpy().transpose()[:,504] - df_close.to_numpy().transpose()[:,0])/df_close.to_numpy().transpose()[:,0]

# Aplicando el algoritmo mapper


In [ ]:
# initializa mapper
mapper = km.KeplerMapper(verbose=1)

KeplerMapper(verbose=1)


In [ ]:
# proyecta los datos a dos dimensiones usando dos transformaciones:
# Coordenada 1 isomap https://en.wikipedia.org/wiki/Isomap
# Coordenada 2 UMAP https://arxiv.org/abs/1802.03426
projected_data = mapper.fit_transform(data, projection=[manifold.Isomap(n_components=100, n_jobs=-1), umap.UMAP(n_components=2,random_state=1)])


..Composing projection pipeline of length 2:
	Projections: Isomap(n_components=100, n_jobs=-1)
		UMAP(random_state=1)
	Distance matrices: False
False
	Scalers: MinMaxScaler()
MinMaxScaler()
..Projecting on data shaped (479, 568)

..Projecting data using: 
	Isomap(n_components=100, n_jobs=-1)


..Scaling with: MinMaxScaler()

..Projecting on data shaped (479, 100)

..Projecting data using: 
	UMAP(random_state=1, verbose=1)

UMAP(random_state=1, verbose=1)
Mon May  1 16:56:58 2023 Construct fuzzy simplicial set
Mon May  1 16:56:58 2023 Finding Nearest Neighbors
Mon May  1 16:56:58 2023 Finished Nearest Neighbor Search
Mon May  1 16:56:58 2023 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Mon May  1 16:57:00 2023 Finished embedding

..Scaling with: MinMaxScaler()



(479, 2)

In [ ]:
# clusterizar los datos using DBSCAN
# Checar los distintos métodos de clusterización https://scikit-learn.org/stable/modules/clustering.html
G = mapper.map(projected_data, data, clusterer=sklearn.cluster.DBSCAN(metric="cosine"))

Mapping on data shaped (479, 568) using lens shaped (479, 2)

Creating 100 hypercubes.

Created 21 edges and 12 nodes in 0:00:00.034402.


In [ ]:
#definir un nombre de archivo para guardar los parámetros de Mapper
fileID = 'projection=' + G['meta_data']['projection'].split('(')[0] + '_' + \
'n_cubes=' + str(G['meta_data']['n_cubes']) + '_' + \
'perc_overlap=' + str(G['meta_data']['perc_overlap']) + '_' + \
'clusterer=' + G['meta_data']['clusterer'].split('(')[0] + '_' + \
'scaler=' + G['meta_data']['scaler'].split('(')[0]


"DBSCAN(metric='cosine')"

In [ ]:
#Visualicemos la gráfica
mapper.visualize(G, 
                path_html="mapper_example_" + fileID + ".html",
                title=fileID,
                custom_tooltips = df_close.columns.to_numpy(),
                color_values = np.log(per_return+1),
                color_function_name = 'Log Percent Returns',
                node_color_function = np.array(['average', 'std', 'sum', 'max', 'min']))

km.draw_matplotlib(G)
plt.show()

Wrote visualization to: mapper_example_projection=UMAP_n_cubes=10_perc_overlap=0.1_clusterer=DBSCAN_scaler=MinMaxScaler.html
no display found. Using non-interactive Agg backend


/usr/local/lib/python3.10/dist-packages/kmapper/visuals.py:344: RuntimeWarning: invalid value encountered in long_scalars
  height = np.floor(((bar / max_bucket_value) * 100) + 0.5)
/usr/local/lib/python3.10/dist-packages/kmapper/visuals.py:345: RuntimeWarning: invalid value encountered in long_scalars
  perc = round((bar / sum_bucket_value) * 100.0, 1)
